In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression,Ridge,Lasso
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor 
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import r2_score,mean_squared_error,make_scorer
import math

In [2]:
turbine_data=pd.read_excel('interview data.xlsx')
turbine_data.columns=['TimeStamp','Generated watts','Exhaust temp','Comp Inlet Temp','Comp IGV angle','Comp discharge pressure','Comp discharge temp']
turbine_data.set_index('TimeStamp',inplace=True)
turbine_data=turbine_data.drop(columns=['Comp Inlet Temp','Comp discharge temp'])

In [3]:
X=turbine_data.loc[:,['Exhaust temp','Comp discharge pressure','Comp IGV angle']]
y=turbine_data.loc[:,'Generated watts']

In [4]:
def accuracyScore(model):
    classifier_pipeline = make_pipeline(StandardScaler(),model)
    y_pred = cross_val_predict(classifier_pipeline,X,y, cv=10)
    return [round(r2_score(y,y_pred),4),round(math.sqrt(mean_squared_error(y,y_pred)),4)]

In [5]:
models={
    'SUPPORT VECTOR REGRESSOR':{'clf_obj':SVR()},
    'RIDGE':{'clf_obj':Ridge()},
    'LASSO':{'clf_obj':Lasso()},
    'LINEAR REGRESSION':{'clf_obj':LinearRegression()},
    'DECISION TREE REGRESSOR':{'clf_obj':DecisionTreeRegressor()}
}

In [6]:
score_board=[]
for model_name,model in models.items():
    clf=model['clf_obj']
    model_score=accuracyScore(clf) 
    score_board.append({'model_name':model_name,'R2':model_score[0],'RMSE':model_score[1]})

In [7]:
score_board_df=pd.DataFrame(score_board,columns=['model_name','R2','RMSE'])
score_board_df.sort_values(by='RMSE',ascending=True,inplace=True)
score_board_df = score_board_df.reset_index(drop=True)
score_board_df

,model_name,R2,RMSE
0,DECISION TREE REGRESSOR,0.9998,0.5754
1,SUPPORT VECTOR REGRESSOR,0.9996,0.7593
2,RIDGE,0.9941,2.8043
3,LINEAR REGRESSION,0.9941,2.8052
4,LASSO,0.9913,3.3904
